In [1]:
import sys
import os
import pandas as pd
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)


from load_inputs.subway_in import load_data as load_data_subway_in,START, END,FILE_NAME
from plotting.TS_analysis import plot_TS
from argparse import Namespace
from examples.benchmark import local_get_args
from plotting.TS_analysis import plot_TS
from constants.paths import FOLDER_PATH

invalid_dates = []
minmaxnorm = True
standardize = False  # Set to True if you want to standardize the data
config = {'model_name': 'STGCN',
         'freq' : '15min',
        'dataset_names':['subway_in'],
        'dataset_for_coverage' : ['subway_in'],
        'target_data': 'subway_in',
        'contextual_kwargs':{'subway_in':{},
                           },
        'step_ahead':1,
        'horizon_step':1,
        }

args = local_get_args(config['model_name'],
                args_init = None,
                dataset_names=config['dataset_names'],
                dataset_for_coverage=config['dataset_for_coverage'],
                modification = config)



coverage_period = pd.date_range(start=START, end=END, freq='15min')[:-1]  # Exclude the last date to match the data
args_out = Namespace(**vars(args))  
args_out.target_data = 'subway_in'
args_out.dataset_for_coverage = ['subway_in']
args_out.dataset_names = ['subway_in']
args_out.contextual_kwargs = {}


ds_in = load_data_subway_in(FOLDER_PATH, coverage_period=coverage_period, invalid_dates=invalid_dates, args=args_out, minmaxnorm=minmaxnorm,standardize=standardize, normalize=True,filename=FILE_NAME,
              tensor_limits_keeper = None)
df_in =  pd.DataFrame(ds_in.U_train[:,:,-1].detach().cpu().numpy(),columns = ds_in.spatial_unit, index = ds_in.tensor_limits_keeper.df_verif_train.iloc[:,-2] )


# Filtered
args_out.contextual_kwargs = {'subway_in':{'quantile_filter_outliers':0.995,
                                        },
                            }

ds_in_filtered = load_data_subway_in(FOLDER_PATH, coverage_period=coverage_period, invalid_dates=invalid_dates, args=args_out, minmaxnorm=minmaxnorm,standardize=standardize, normalize=True,filename=FILE_NAME,
              tensor_limits_keeper = None)
df_in_filtered =  pd.DataFrame(ds_in_filtered.U_train[:,:,-1].detach().cpu().numpy(),columns = [f"filtered_{c}" for c in ds_in_filtered.spatial_unit], index = ds_in_filtered.tensor_limits_keeper.df_verif_train.iloc[:,-2] )
# ---

df_out_concat = pd.concat([df_in.iloc[:,:10], df_in_filtered.iloc[:,:10]], axis=1)

plot_TS(df_out_concat.iloc[:2000,:],bool_show = True, height = 800)

/root/anaconda3/envs/pytorch-2.0.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module xgboost
Training and Hyper-parameter tuning with Ray is not possible
   Load data from: //home/rrochas/../../data/rrochas/prediction_validation/subway_in/subway_in.csv
   Load data from: //home/rrochas/../../data/rrochas/prediction_validation/subway_in/subway_in.csv
OUTLIER QUANTILE REMOVED


Loading BokehJS ...

figure(id='p1001', ...)